In [60]:
import pandas as pd
import pickle

from sklearn.model_selection import TimeSeriesSplit

from typing import Dict

In [61]:
# df = pd.read_pickle('/data/edmitrie/alfa_boosters/net_atrs_first_net/part-00009.parquet.pckl')

# from sklearn.preprocessing import LabelEncoder

# lb_enc.fit(df[df['target'] !='nan']['target'].unique())
# lb_enc = LabelEncoder()

# lb_enc.fit(df[df['target'] !='nan']['target'].unique())

In [62]:
lb_encoder = pickle.load(open('target_enc_2_net.pickle','rb'))

In [63]:
df = pd.read_pickle('/data/edmitrie/alfa_boosters/net_atrs_first_net/part-00009.parquet.pckl')

In [64]:
# df = pd.read_pickle('/data/edmitrie/alfa_boosters/df_atr_1_net.pickle')

In [65]:
df = df[df['target'] != 'nan']

In [66]:
df['target'] = lb_encoder.transform(df['target'])

In [67]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [68]:
# df['timestamp']

In [69]:
%%time
df = df.sort_values(by='timestamp')

CPU times: user 5.72 s, sys: 233 ms, total: 5.95 s
Wall time: 5.93 s


In [70]:
# train_test_df.groupby('client_pin').size().hist(log=True)

NameError: name 'train_test_df' is not defined

In [72]:
# train_test_df.groupby('client_pin').size().sort_values().head(n=70)

In [ ]:
1, 2, 3, 4, 5, 6

## Формирование валидационной,тестовой,трейн выборок

In [73]:
import numpy as np

In [74]:
gr_cl = df.groupby('client_pin').size()

In [75]:
# use_clnts = pd.Series(gr_cl[gr_cl > 3].index)
use_clnts = pd.Series(gr_cl[gr_cl > 2].index)

In [76]:
train_test_df = df.merge(use_clnts, on='client_pin', how='inner')

In [77]:
train_test_df.sort_values(by='timestamp', inplace=True)

In [78]:
train_test_df['weekday'] = train_test_df['timestamp'].apply(lambda x: x.weekday())
train_test_df['day'] = train_test_df['timestamp'].apply(lambda x: x.day)
train_test_df['month'] = train_test_df['timestamp'].apply(lambda x: x.month)
train_test_df['hour'] = train_test_df['timestamp'].apply(lambda x: x.hour)

In [102]:
train_test_df['diff_time_1_0'] = train_test_df.groupby('client_pin')['timestamp'].shift(1)

In [103]:
train_test_df['diff_time_1_0'] = (train_test_df['timestamp'] - train_test_df['diff_time_1_0'])/np.timedelta64(1,'m')

In [104]:
train_test_df['diff_time_1_0']

0                  NaN
46                 NaN
177                NaN
277                NaN
633                NaN
              ...     
497795     8805.361100
294674      370.573050
479635    16027.860317
288104      488.707000
398492    95545.743767
Name: diff_time_1_0, Length: 498052, dtype: float64

In [482]:
from sklearn.preprocessing import MinMaxScaler

In [484]:
scaler = MinMaxScaler()

In [487]:
train_test_df['diff_time_1_0'] = scaler.fit_transform(train_test_df[['diff_time_1_0']])

In [576]:
train_test_df['diff_time_1_0'].fillna(0, inplace=True)

In [578]:
train_df = train_test_df.groupby('client_pin').apply(lambda x: x.iloc[:-1])

In [579]:
test_df = train_test_df

In [580]:
# train_df = pd.merge(train_test_df, test_df['session_id'], how='left', indicator=True)

In [581]:
# train_df = train_df[train_df['_merge'] == 'left_only']

In [582]:
# train_df.shape, test_df.shape, valid_df.shape, df.shape

In [747]:
from sklearn.preprocessing import MinMaxScaler

## Формирование сэмплов под нейронку

In [768]:
def truncate(x, num_last_transactions=750):
    return x.values.transpose()[:, -num_last_transactions:].tolist()


def transform_transactions_to_sequences(transactions_frame: pd.DataFrame,atrs,
                                        num_last_transactions=300) -> pd.DataFrame:
    return transactions_frame \
        .groupby(['client_pin'])[atrs] \
        .apply(lambda x: truncate(x, num_last_transactions=num_last_transactions)) \
        .reset_index().rename(columns={0: 'sequences'})


In [584]:
def pad_sequence(array, max_len) -> np.array:
    add_zeros = int(max_len - len(array[0]))
    return np.array([list(x) + [0] * add_zeros for x in array])



def create_padded_buckets(frame_of_sequences: pd.DataFrame, bucket_info: Dict[int, int],
                          save_to_file_path=None, has_target=True):
    frame_of_sequences['bucket_idx'] = frame_of_sequences.sequence_length.map(bucket_info)
        
    
#     print(frame_of_sequences.sequence_length.max(), frame_of_sequences.sequence_length.min())
        
    padded_seq = []
    targets = []
    lengths = []
    client_pin = []
    
#     print(frame_of_sequences.session_id.unique().shape)
#     print(frame_of_sequences.bucket_idx.isnull().sum())

    
    from tqdm import tqdm

    for size, bucket in frame_of_sequences.groupby('bucket_idx'):
        padded_sequences = bucket.sequences.apply(lambda x: pad_sequence(x, size)).values
        padded_sequences = np.array([np.array(x) for x in padded_sequences])
        padded_seq.append(padded_sequences)
        
        if has_target:
            targets.append(bucket.target.values)
            
#         print(bucket.sequence_length.shape)
            
        lengths.append(bucket.sequence_length.values)
        client_pin.append(bucket.client_pin.values)        
    

    frame_of_sequences.drop(columns=['bucket_idx'], inplace=True)
    
    
    if has_target:
        dict_result = {
            'padded_sequences':np.array(padded_seq),
            'targets':np.array(targets),
            'lengths': np.array(lengths),
            'client': np.array(client_pin),
        }
    else:
         dict_result = {
            'padded_sequences':np.array(padded_seq),
            'lengths': np.array(lengths),
            'client': np.array(client_pin),
        }

    if save_to_file_path:
        with open(save_to_file_path, 'wb') as f:
            pickle.dump(dict_result, f)
    return dict_result


def create_buckets_from_transactions(df, atrs, file_name, submit=False):   
    import tqdm
    import os
    
    train_df = df
    
    
    if not submit:
        target = df.groupby('client_pin')[['target','session_id', 'timestamp']].last().reset_index()

    #     return target

        train_df = df.merge(target[['session_id']], on='session_id', how='left', indicator=True)
        train_df = train_df[train_df['_merge'] == 'left_only']
        
        train_df.drop(columns='_merge', inplace=True)
        
    seq = transform_transactions_to_sequences(train_df, atrs)
    
#     print(seq)
        
    seq['sequence_length'] = seq.sequences.apply(lambda x: len(x[0]))
    
    if not submit:                
        seq = seq.merge(target[['client_pin', 'target']], how='left', on='client_pin')
    
    mapping_seq_len_to_padded_len = pickle.load(open('buckets_info.pkl', 'rb'))
    
    if not submit:
        processed_fragment =  create_padded_buckets(seq, mapping_seq_len_to_padded_len, has_target=True)    
    else:
        processed_fragment =  create_padded_buckets(seq, mapping_seq_len_to_padded_len, has_target=False)    
    res = pd.DataFrame(processed_fragment)
    res.to_pickle(file_name)
    
    return res

In [585]:
def get_data_for_net(file_name):
    print(file_name)

    df = pd.read_pickle(file_name)
    
    print(file_name)
    
    df = df[df['target'] != 'nan']
    
    import pickle
    
    lb_encoder = pickle.load(open('target_enc_2_net.pickle','rb'))
    
    df['target'] = lb_encoder.transform( df['target'])
    
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.sort_values(by='timestamp', inplace=True)
    
    atrs = [col for col in df.columns if col not in ['client_pin', 'session_id', 'target', 'timestamp']]
    
    last_name = file_name.split('/')[-1]
    
    print('приступаю к сабмит файлу')
    
    create_buckets_from_transactions(df, f'/data/edmitrie/alfa_boosters/net_atrs_2_net/{last_name}_submit', submit=True)
    
    print('сабмит сохранил')
    
    gr_cl = df.groupby('client_pin').size()
    use_clnts = pd.Series(gr_cl[gr_cl > 2].index)
    
    train_test_df = df.merge(use_clnts, on='client_pin', how='inner')
    
    train_df = train_test_df.groupby('client_pin').apply(lambda x: x.iloc[:-1]).reset_index(drop='True')
    test_df = df
    
    train_df.sort_values(by='timestamp', inplace=True)
    test_df.sort_values(by='timestamp', inplace=True)

    create_buckets_from_transactions(train_df, f'/data/edmitrie/alfa_boosters/net_atrs_2_net/{last_name}_train')
    
    print('сохранил трейн')
    
    create_buckets_from_transactions(test_df, f'/data/edmitrie/alfa_boosters/net_atrs_2_net/{last_name}_test')  
    print('сохранил тест')

In [769]:
def get_data_for_net(file_name, cat_atrs, num_atrs):
    atrs = cat_atrs + num_atrs
    print(file_name)

    df = pd.read_pickle(file_name)
    
    print(file_name)
    
    df = df[df['target'] != 'nan']
    
    import pickle
    
    lb_encoder = pickle.load(open('target_enc_2_net.pickle','rb'))
    
    df['target'] = lb_encoder.transform( df['target'])
    
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.sort_values(by='timestamp', inplace=True)
        
    last_name = file_name.split('/')[-1]
    
    print('приступаю к сабмит файлу')
    
    df['weekday'] = df['timestamp'].apply(lambda x: x.weekday())
    df['day'] = df['timestamp'].apply(lambda x: x.day)
    df['month'] = df['timestamp'].apply(lambda x: x.month)
    df['hour'] = df['timestamp'].apply(lambda x: x.hour)
    
    df['diff_time_1_0'] = df.groupby('client_pin')['timestamp'].shift(1)
    df['diff_time_1_0'] = (df['timestamp'] - df['diff_time_1_0'])/np.timedelta64(1,'m')
    
    df['diff_time_1_0'].fillna(0, inplace=True)
        
    scaler = MinMaxScaler()
    
    df['diff_time_1_0'] = scaler.fit_transform(df[['diff_time_1_0']])

    create_buckets_from_transactions(df, atrs, f'/data/edmitrie/alfa_boosters/net_atrs_2_net/{last_name}_submit', submit=True)
    
    print('сабмит сохранил')
    
    gr_cl = df.groupby('client_pin').size()
    use_clnts = pd.Series(gr_cl[gr_cl > 2].index)
    
    train_test_df = df.merge(use_clnts, on='client_pin', how='inner')
    
    train_df = train_test_df.groupby('client_pin').apply(lambda x: x.iloc[:-1]).reset_index(drop='True')
    test_df = df
    
    train_df.sort_values(by='timestamp', inplace=True)
    test_df.sort_values(by='timestamp', inplace=True)

#     create_buckets_from_transactions(train_df, f'/data/edmitrie/alfa_boosters/net_atrs_2_net/{last_name}_train')

    get_augmenteded_samples(train_df, cat_atrs, num_atrs, max_seq=300, file_to_save=f'/data/edmitrie/alfa_boosters/net_atrs_2_net/{last_name}_train')
    
    print('сохранил трейн')
    
    create_buckets_from_transactions(test_df, atrs, f'/data/edmitrie/alfa_boosters/net_atrs_2_net/{last_name}_test')  
    print('сохранил тест')

In [749]:
def get_augmenteded_samples(df, cat_atrs, num_atrs, max_seq, file_to_save=None):
    def get_samples(df, window = 1, max_seq = 20):    
        samples_n = len(df)

        groups_number = samples_n // window

        sec_seqs = [df.iloc[:(i+1)*window].iloc[-max_seq:].values for i in range(groups_number)]

        if (samples_n % window) !=0:
            sec_seqs.append(df.iloc[-max_seq:].values)

        return sec_seqs
    
    df.reset_index(inplace=True, drop=True)
    
    atrs = cat_atrs + num_atrs
    
    agg_df = df.groupby('client_pin')[atrs].apply(lambda x: get_samples(x, max_seq=max_seq))
    
    agg_df = pd.DataFrame(agg_df.explode(), columns=['sequences'])
        
    agg_df['target'] = agg_df.sequences.apply(lambda x: x[-1][len(cat_atrs) - 1])
    agg_df['sequences'] = agg_df.sequences.apply(lambda x: x[:-1])
    agg_df['sequence_length'] = agg_df.sequences.apply(lambda x: len(x))
    agg_df['sequences'] = agg_df['sequences'].apply(lambda x:[[el[i] for el in x] for i in range(len(atrs))])
        
#     agg_df['sequence_length'] = agg_df.sequences.apply(lambda x: len(x))
    agg_df.reset_index(inplace=True)

    mapping_seq_len_to_padded_len = pickle.load(open('buckets_info.pkl', 'rb'))

    df = create_padded_buckets(agg_df, mapping_seq_len_to_padded_len, has_target=True, save_to_file_path=file_to_save)
    
    return df

In [ ]:
[[1,], [2], [1], [2]], [[1, 2, 3, 4], [...]]

In [736]:
%%time
atrs = ['hour', 'weekday', 'day', 'diff_time_1_0', 'target']
cat_atrs = ['hour', 'weekday', 'day', 'target']
num_atrs = ['diff_time_1_0']

test_df_net = create_buckets_from_transactions(test_df, cat_atrs + num_atrs, 'chunks_data/processed_test_target_2_net.pkl')
train_df_net = get_augmenteded_samples(train_df, cat_atrs, num_atrs, max_seq=200, file_to_save='chunks_data/processed_train_target_2_net.pkl' )

# train_df_net = create_buckets_from_transactions(train_df, 'chunks_data/processed_train_target_2_net.pkl')
# valid_df_net = create_buckets_from_transactions(valid_df, 'chunks_data/processed_valid_2_net.pkl')

KeyboardInterrupt: 

In [770]:
import os

DIR = '/data/edmitrie/alfa_boosters/net_atrs_first_net/'

for file in os.listdir(DIR):
    get_data_for_net(DIR + file, cat_atrs, num_atrs)

/data/edmitrie/alfa_boosters/net_atrs_first_net/part-00009.parquet.pckl
/data/edmitrie/alfa_boosters/net_atrs_first_net/part-00009.parquet.pckl
приступаю к сабмит файлу


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:51: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:52: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:53: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


сабмит сохранил


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local

сохранил трейн
сохранил тест
/data/edmitrie/alfa_boosters/net_atrs_first_net/part-00000.parquet.pckl
/data/edmitrie/alfa_boosters/net_atrs_first_net/part-00000.parquet.pckl
приступаю к сабмит файлу


...

сохранил трейн
сохранил тест
/data/edmitrie/alfa_boosters/net_atrs_first_net/part-00008.parquet.pckl
/data/edmitrie/alfa_boosters/net_atrs_first_net/part-00008.parquet.pckl
приступаю к сабмит файлу


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:51: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:52: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:53: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


сабмит сохранил


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local

сохранил трейн
сохранил тест


In [ ]:
%%time
# get_data_for_net('/data/edmitrie/alfa_boosters/net_atrs_first_net/part-00008.parquet.pckl')

In [ ]:
# seq = transform_transactions_to_sequences(test_df, atrs)


In [ ]:
# test_df_net = pd.read_pickle('chunks_data/processed_test_2_net.pkl')

## Моделирование

In [589]:
import torch
import pandas as pd

from torch import nn

In [590]:
# test_df_net = pd.read_pickle('chunks_data/processed_test_2_net.pkl')
# train_df_net = pd.read_pickle('chunks_data/processed_train_2_net.pkl')
# # valid_df_net = pd.read_pickle('chunks_data/processed_valid_2_net.pkl')

In [591]:
def batches_generator(list_of_paths,transaction_features, num_atrs, batch_size=32, shuffle=False, is_infinite=False,
                      verbose=False, device=None, output_format='torch', is_train=True):
    while True:
        if shuffle:
            np.random.shuffle(list_of_paths)

        for path in list_of_paths:
            if verbose:
                print(f'reading {path}')
            
#             print(path)
            
            data = pd.read_pickle(path)

            padded_sequences, targets, lengths = data['padded_sequences'], data['targets'], data['lengths']
            
            indices = np.arange(len(targets))

            if shuffle:
                np.random.shuffle(indices, )
                
                padded_sequences = padded_sequences[indices]
                targets = targets[indices]
                lengths = lengths[indices]

            for idx in range(len(padded_sequences)):

                bucket = padded_sequences[idx]
                
                if is_train:
                    target = targets[idx]
                    length = lengths[idx]
                
                for jdx in range(0, len(bucket), batch_size):
                    batch_sequences = bucket[jdx: jdx + batch_size]
                    
                                        
                    if is_train:
                        batch_targets = target[jdx: jdx + batch_size]
                        batch_length = length[jdx: jdx + batch_size]
                        
                    batch_sequences_last = [torch.LongTensor(batch_sequences[:, i]).to(device)
                                       for i in range(len(transaction_features))]
                    
                    batch_sequences_last.append(torch.FloatTensor(batch_sequences[:, len(transaction_features):]).to(device))
                                    
                    if is_train:
                        yield dict(transactions_features=batch_sequences_last,
                                   label=torch.LongTensor(batch_targets).to(device),
                                   lengths=torch.LongTensor(batch_length).to(device)
                                  )
                    else:
                        yield dict(transactions_features=batch_sequences_last,
                                   product=torch.LongTensor(batch_products).to(device),
                                   lengths=torch.LongTensor(data['lengths']).to(device)
                                   )
        if not is_infinite:
            break

In [592]:
from sklearn.metrics import f1_score
import os

In [593]:
device='cuda'

In [782]:
DIR = '/data/edmitrie/alfa_boosters/net_atrs_2_net/'

In [783]:
[DIR + el for el in os.listdir(DIR) if 'train' in el]

['/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00008.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00009.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00000.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00001.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00003.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00002.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00004.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00005.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00006.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00007.parquet.pckl_train']

In [732]:
pickle.load(open('/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00008.parquet.pckl_train','rb'))

,padded_sequences,targets,lengths,client
0,"[[[2, 5, 5, 9, 2]], [[3, 5, 3, 3, 3]], [[2, 0,...","[5, 3, 3, 1, 7, 5, 5, 2, 3, 5, 5, 2, 9, 5, 5, ...","[5, 5, 1, 2, 3, 1, 1, 3, 5, 4, 2, 2, 4, 3, 4, ...","[0013017a0c3a4d52814367706a6fb4e6, 003a8390e06..."
1,"[[[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]], [[...","[5, 5, 3, 3, 0, 6, 5, 1, 6, 2, 5, 3, 5, 5, 9, ...","[13, 8, 12, 7, 11, 10, 12, 12, 8, 13, 12, 6, 8...","[002f4de4ccba696382a2c045942adba5, 0109101437b..."
2,"[[[1, 3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 3, 9, 9, 0...","[1, 8, 1, 5, 5, 5, 3, 5, 5, 5, 7, 9, 7, 5, 1, ...","[14, 17, 15, 19, 17, 20, 20, 18, 20, 19, 19, 1...","[00488fd7358b29b7ab9924f5509b4645, 009da692f13..."
3,"[[[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5...","[5, 5, 9, 2, 0, 5, 5, 8, 3, 7, 3, 6, 5, 0, 5, ...","[21, 24, 25, 27, 22, 26, 24, 24, 24, 25, 23, 2...","[006c4ffd41de806fdd448aa8da2de4fb, 0291a8121e0..."
4,"[[[2, 9, 2, 3, 9, 3, 1, 1, 2, 9, 5, 9, 5, 1, 9...","[9, 9, 5, 5, 9, 5, 8, 5, 7, 5, 5, 5, 5, 5, 5, ...","[30, 29, 31, 34, 31, 29, 30, 33, 32, 34, 30, 3...","[006da09fb1c8c802b19155fe326498aa, 01821b37f01..."
5,"[[[2, 5, 2, 2, 5, 2, 5, 2, 2, 5, 5, 5, 2, 2, 2...","[5, 4, 5, 0, 5, 5, 3, 5, 3, 5, 2, 5, 9, 5, 2, ...","[37, 39, 43, 41, 39, 39, 38, 41, 38, 42, 41, 4...","[0011dab5a085273c80be9282b1c0c4de, 005fa922eb8..."
6,"[[[1, 1, 9, 2, 1, 5, 1, 2, 9, 1, 1, 1, 1, 1, 5...","[5, 6, 3, 5, 5, 2, 7, 9, 3, 5, 5, 5, 5, 5, 9, ...","[48, 44, 46, 44, 45, 50, 50, 46, 47, 47, 49, 4...","[0183363864a0d032dc3e35f4b798b731, 030b226252b..."
7,"[[[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5...","[5, 8, 5, 5, 5, 5, 9, 5, 3, 2, 5, 5, 3, 8, 8, ...","[58, 58, 56, 53, 51, 51, 52, 55, 58, 52, 57, 5...","[013214d1fef11a990d31fbb86713106f, 01a1c3d59bf..."
8,"[[[5, 5, 5, 5, 9, 8, 5, 6, 5, 5, 5, 5, 5, 4, 5...","[5, 5, 9, 5, 5, 5, 8, 6, 5, 9, 5, 9, 5, 5, 5, ...","[60, 64, 61, 64, 62, 60, 61, 60, 59, 59, 62, 6...","[0022b12a2ec1645c8ecee1e77191e75b, 005b433727c..."
9,"[[[5, 9, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5...","[5, 5, 9, 3, 3, 9, 9, 5, 7, 0, 5, 5, 9, 7, 5, ...","[67, 72, 71, 72, 66, 68, 68, 66, 71, 70, 67, 6...","[0099a87930be9c04d0a4a26877023f13, 00fa0652549..."


In [784]:
dataset_train = [DIR + el for el in os.listdir(DIR) if 'train' in el]
# dataset_valid = ['chunks_data/processed_test_target_2_net.pkl']
dataset_valid =  [DIR + el for el in os.listdir(DIR) if 'test' in el]

In [785]:
dataset_valid

['/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00008.parquet.pckl_test',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00009.parquet.pckl_test',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00000.parquet.pckl_test',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00001.parquet.pckl_test',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00003.parquet.pckl_test',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00002.parquet.pckl_test',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00004.parquet.pckl_test',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00005.parquet.pckl_test',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00006.parquet.pckl_test',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00007.parquet.pckl_test']

In [726]:
dataset_train

['/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00008.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00009.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00000.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00001.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00003.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00002.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00004.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00005.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00006.parquet.pckl_train',
 '/data/edmitrie/alfa_boosters/net_atrs_2_net/part-00007.parquet.pckl_train']

In [723]:
# dataset_train = ['chunks_data/processed_train_target_2_net.pkl']
# dataset_valid = ['chunks_data/processed_test_target_2_net.pkl']

In [724]:
from torch import nn

In [725]:
import time

time.sleep(1)

In [794]:
def train_epoch(model, optimizer, cat_atrs, num_atrs, dataset_train, atrs, batch_size=64, shuffle=True,
                print_loss_every_n_batches=10, device=None):
    train_generator = batches_generator(dataset_train, cat_atrs, num_atrs, batch_size=batch_size, shuffle=shuffle,
                                        device=device, is_train=True, output_format='torch')
    
    loss_function = nn.CrossEntropyLoss()

    num_batches = 1
    running_loss = 0.0

    model.train()

    for i, batch in enumerate(train_generator):
#         print(batch['transactions_features'][0].shape)
        
        
        _, output = model(batch['transactions_features'], batch['lengths'])
        

        batch_loss = loss_function(output, batch['label'])
        
        if i % print_loss_every_n_batches == 0:
#             val_generator = batches_generator(dataset_valid, atrs, batch_size=batch_size, shuffle=False,
#                                       device=device, is_train=True, output_format='torch')
            model_os = []
            targets_l = []
            
            valid_dataset = pd.read_pickle(dataset_valid[1])
                        
            for (seqs, targets, lengths) in zip(valid_dataset['padded_sequences'], valid_dataset['targets'], valid_dataset['lengths']):
                batch_sequences = [torch.LongTensor(seqs[:, i]).to(device)
                                       for i in range(len(cat_atrs))]
                
                batch_sequences.append(torch.FloatTensor(seqs[:, len(cat_atrs):]).to(device))
                
                
                                    
                lengths = torch.LongTensor(lengths).to(device)
    
                _, logits = model(batch_sequences, lengths)
    
                model_os.append(logits.argmax(axis=-1).detach().cpu().numpy().flatten())
                targets_l.append(targets)
            
#             for batch_t in val_generator:
                
#                 _, output = model(batch_t['transactions_features'], batch_t['lengths'])
#                 metrics.append(f1_score(batch_t['label'].detach().cpu().numpy().flatten(), output.argmax(axis=-1).detach().cpu().numpy().flatten(), average='macro'))
                
            print('Метрика на тесте', f1_score(np.concatenate(model_os), np.concatenate(targets_l), average='macro'))


        batch_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += batch_loss
        
        if num_batches % print_loss_every_n_batches == 0:
            print(f'Training loss after {num_batches} batches: {running_loss / num_batches}', end='\r')
        
        num_batches += 1
    
    print(f'Training loss after epoch: {running_loss / num_batches}', end='\r')

In [795]:
class TransactionsRnn(nn.Module):
    def __init__(self, embedding_projections, transaction_features, num_atrs, rnn_units=256, top_classifier_units=32, classes=10):
        super(TransactionsRnn, self).__init__()
        
        self.embedding_projections = {feature:embedding_projections[feature] for feature in transaction_features}
                
        self._transaction_cat_embeddings = nn.ModuleList([self._create_embedding_projection(*embedding_projections[feature]) 
                                                          for feature in transaction_features])
                                        
#         self._gru = nn.GRU(input_size=sum([embedding_projections[x][1] for x in embedding_projections]),
#                              hidden_size=rnn_units, batch_first=True, bidirectional=False)

        self.transaction_features = transaction_features

        self._gru = nn.LSTM(input_size=sum([self.embedding_projections[x][1] for x in self.embedding_projections]) + len(num_atrs),
                              hidden_size=rnn_units, batch_first=True, bidirectional=False)
        
        self._hidden_size = rnn_units
                
        self._top_classifier = nn.Linear(in_features=rnn_units, 
                                         out_features=top_classifier_units)
        
        self._intermediate_activation = nn.ReLU()
        
        self._head = nn.Linear(in_features=top_classifier_units, out_features=classes)
        
        self.transaction_atrs = transaction_features
        self.num_atrs = num_atrs
    
    def forward(self, transactions_cat_features, lengths):   
        num_vector = transactions_cat_features[len(self.transaction_atrs)].permute(0, 2, 1)
        
        embeddings = [embedding(transactions_cat_features[i]) for i, embedding in enumerate(self._transaction_cat_embeddings)]
        
        embeddings.append(num_vector)
        
        concated_embeddings = torch.cat(embeddings, dim=-1)
        
#         print(concated_embeddings.shape)
        
#         print(concated_embeddings.shape)
        
#         print(concated_embeddings.shape)
#         print(concated_embeddings.shape, lengths.shape)
        
        packed_inputs = torch.nn.utils.rnn.pack_padded_sequence(concated_embeddings, lengths, batch_first=True, enforce_sorted=False)
        
#         print(concated_embeddings.shape)
            
        final_hiddens = self._gru(packed_inputs)[1][0]
                        
        final_hiddens = final_hiddens.view(concated_embeddings.size(0), -1)
                                                   
        classification_hidden = self._top_classifier(final_hiddens)
        activation = self._intermediate_activation(classification_hidden)
        
        logits = self._head(activation)
        
        return [final_hiddens, logits]
    
    @classmethod
    def _create_embedding_projection(cls, cardinality, embed_size, padding_idx=0):
        # +1 категория other ещё
        return nn.Embedding(num_embeddings=cardinality + 1, embedding_dim=embed_size, padding_idx=padding_idx)

In [705]:
import torch

In [706]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [707]:
device = 'cuda'

In [708]:
def emb_sz_rule(n_cat:int)->int: return min(600, round(1.6 * n_cat**0.56))

embedding_projections = {'target':(10, emb_sz_rule(10)), 'weekday':(7, emb_sz_rule(7)),  'day':(31, emb_sz_rule(31)), 'hour':(24, emb_sz_rule(24)), 'month':(12,emb_sz_rule(12))}

In [709]:
steps = 20

In [710]:
num_epochs = 200
train_batch_size = 512
val_batch_szie = 256

In [711]:
import numpy as np

In [712]:
from sklearn.metrics import f1_score

In [713]:
atrs = ['hour', 'target']

In [714]:
cat_atrs, num_atrs

(['hour', 'weekday', 'day', 'target'], ['diff_time_1_0'])

In [791]:
model = TransactionsRnn(embedding_projections, cat_atrs, num_atrs)
model = model.to(device)

SEED = 3

optimizer = torch.optim.Adam(lr=1e-3, params=model.parameters())

In [792]:
model

TransactionsRnn(
  (_transaction_cat_embeddings): ModuleList(
    (0): Embedding(25, 9, padding_idx=0)
    (1): Embedding(8, 5, padding_idx=0)
    (2): Embedding(32, 11, padding_idx=0)
    (3): Embedding(11, 6, padding_idx=0)
  )
  (_gru): LSTM(32, 256, batch_first=True)
  (_top_classifier): Linear(in_features=256, out_features=32, bias=True)
  (_intermediate_activation): ReLU()
  (_head): Linear(in_features=32, out_features=10, bias=True)
)

In [796]:
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}')
    
    np.random.seed(SEED)
    
    train_epoch(model, optimizer, cat_atrs, num_atrs, dataset_train, atrs,  batch_size=train_batch_size, 
                shuffle=True, print_loss_every_n_batches=500, device=device)
#     val_roc_auc = eval_model(model, dataset_val, batch_size=val_batch_szie, device=device)
#     es(val_roc_auc, model)

#     if es.early_stop:

#         print('Early stopping reached. Stop training...')
#         break
#     torch.save(model.state_dict(), os.path.join(path_to_checkpoints, f'epoch_{epoch+1}_val_{val_roc_auc:.3f}.pt'))
    
#     train_roc_auc = eval_model(model, dataset_train, batch_size=val_batch_szie, device=device)
#     print(f'Epoch {epoch+1} completed. Train roc-auc: {train_roc_auc}, Val roc-auc: {val_roc_auc}')

Starting epoch 1
Метрика на тесте 0.19568330188046457
Метрика на тесте 0.30006821727935943353303670883179
Метрика на тесте 0.3411202131124661.273321270942688
Метрика на тесте 0.3719765706595035.239812970161438
Метрика на тесте 0.3748954681767529.2350279092788696
Метрика на тесте 0.3828662940343896.235181212425232
Метрика на тесте 0.3816790061536331521725594997406
Метрика на тесте 0.386750585108706632113021612167358
Метрика на тесте 0.384421244622703662048617601394653
Метрика на тесте 0.3767046303878807.2004958391189575
Метрика на тесте 0.375244172880676242001031637191772
Метрика на тесте 0.3961130735423766.197066068649292
Метрика на тесте 0.36979034999265725193635106086731
Метрика на тесте 0.378148442967063571906646490097046
Метрика на тесте 0.3844247971229333.1883398294448853
Метрика на тесте 0.35215963491226837188040018081665
Метрика на тесте 0.389395719686523031906232833862305
Метрика на тесте 0.37844361753350275187757134437561
Метрика на тесте 0.3760252312311891.1872063875198364
Ме

KeyboardInterrupt: 

In [504]:
train_df_net['targets'].shape

(3,)

In [515]:
max(train_df_net['targets'][1])

9.0

In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = 'cpu'

In [260]:
model.to(device)

TransactionsRnn(
  (_transaction_cat_embeddings): ModuleList(
    (0): Embedding(11, 6, padding_idx=0)
  )
  (_gru): GRU(6, 128, batch_first=True)
  (_top_classifier): Linear(in_features=128, out_features=32, bias=True)
  (_intermediate_activation): ReLU()
  (_head): Linear(in_features=32, out_features=11, bias=True)
)

In [780]:
model_os = []
targets_l = []

for sequnces, targets, lengths in zip(test_df_net['padded_sequences'], test_df_net['targets'], test_df_net['lengths']):
    batch_sequences = torch.LongTensor(sequnces).to(device)
    lengths = torch.LongTensor(lengths).to(device)
    
    logits = model(batch_sequences, lengths)
    
    model_os.append(logits.argmax(axis=-1).detach().cpu().numpy().flatten())
    targets_l.append(targets)

RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_index_select

In [139]:
outputs = np.concatenate(model_os)

In [140]:
targets = np.concatenate(targets_l)

In [141]:
pd.DataFrame(classification_report(targets, outputs, output_dict=True)).T

/home/edmitrie/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
0,0.333333,0.038462,0.068966,260.000000
1,0.438849,0.204698,0.279176,298.000000
2,0.407407,0.028796,0.053790,382.000000
3,0.557258,0.679449,0.612317,1017.000000
4,0.000000,0.000000,0.000000,54.000000
5,0.588089,0.809034,0.681091,3210.000000
6,0.540541,0.058140,0.104987,344.000000
8,0.421053,0.455696,0.437690,316.000000
9,0.340741,0.224939,0.270987,409.000000
10,0.455310,0.372633,0.409844,1162.000000


In [ ]:
numpy.concatenate

In [125]:
test_df_net['padded_sequences']

array([[[ 1.38508677e-01,  9.51670259e-02,  7.62902126e-02,
         -9.14834216e-02,  0.00000000e+00],
        [-3.58143896e-02,  3.48560989e-01,  3.76887798e-01,
         -3.72102708e-02,  0.00000000e+00],
        [-9.99994278e-01,  9.91091907e-01,  9.98991191e-01,
         -9.99996662e-01,  0.00000000e+00],
        ...,
        [-8.00324798e-01, -2.36761430e-03, -1.84023124e-03,
         -8.11739862e-01,  0.00000000e+00],
        [-9.99752820e-01,  2.20804080e-01,  2.00806692e-01,
         -9.99979198e-01,  0.00000000e+00],
        [-2.19531164e-01, -4.22228314e-02, -2.42895521e-02,
         -9.14154872e-02,  0.00000000e+00]],

       [[ 1.17300853e-01, -6.57980144e-01,  9.19858664e-02,
         -9.67029393e-01, -9.87110257e-01],
        [ 7.23170400e-01, -9.94472265e-01,  8.18345070e-01,
         -9.99246240e-01, -9.98692095e-01],
        [ 6.78703308e-01, -7.73295760e-01,  9.95796621e-01,
         -9.96250451e-01, -4.41340417e-01],
        ...,
        [-7.68387970e-03, -9.5823371

In [117]:
targets.shape

(715,)

In [118]:
batch_sequences = torch.FloatTensor(sequnces).to(device)
lengths = torch.LongTensor(lengths).to(device)

In [119]:
logits = model(batch_sequences, lengths)

In [120]:
from sklearn.metrics import classification_report

In [121]:
pd.DataFrame(classification_report(targets, logits.argmax(axis=-1).detach().cpu().numpy().flatten(), output_dict=True)).T

/home/edmitrie/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
0,0.500000,0.041667,0.076923,24.000000
1,0.230769,0.096774,0.136364,31.000000
2,0.250000,0.025000,0.045455,40.000000
3,0.562500,0.658537,0.606742,123.000000
4,0.000000,0.000000,0.000000,2.000000
5,0.581609,0.826797,0.682861,306.000000
6,0.666667,0.071429,0.129032,28.000000
8,0.428571,0.240000,0.307692,25.000000
9,0.454545,0.285714,0.350877,35.000000
10,0.525641,0.405941,0.458101,101.000000


In [109]:
pd.Series(targets).value_counts()

5     458
10    174
3     143
2      76
1      69
9      42
8      32
6      28
0      28
4      11
dtype: int64

In [99]:
f1_score(targets, logits.argmax(axis=-1).detach().cpu().numpy().flatten(), average='macro')

0.24481540761293985

In [173]:
batch_sequences

RuntimeError: cuda runtime error (710) : device-side assert triggered at /pytorch/aten/src/THC/THCCachingHostAllocator.cpp:278

In [164]:
lengths

tensor([4, 5, 5, 3, 3, 3, 5, 4, 5, 3, 5, 4, 4, 4, 5, 5, 4, 5, 5, 4, 5, 5, 4, 3,
        5, 3, 5, 3, 5, 3, 3, 5, 4, 5, 3, 4, 4, 4, 5, 4, 4, 5, 5, 5, 5, 4, 5, 3,
        4, 3, 3, 3, 3, 5, 4, 4, 5, 4, 4, 3, 3, 4, 5, 5, 4, 5, 5, 3, 5, 4, 5, 3,
        5, 3, 3, 3, 5, 4, 3, 4, 4, 3, 3, 3, 4, 3, 4, 3, 4, 3, 5, 5, 4, 3, 4, 3,
        4, 3, 4, 4, 4, 4, 4, 3, 5, 5, 4, 5, 4, 5, 5, 3, 3, 4, 5, 3, 4, 4, 5, 4,
        5, 3, 4, 5, 4, 5, 4, 4, 4, 4, 3, 3, 3, 4, 4, 3, 4, 5, 3, 4, 5, 3, 4, 4,
        5, 3, 3, 5, 4, 4, 5, 4, 4, 3, 5, 4, 5, 3, 3, 4, 3, 4, 4, 5, 4, 4, 5, 4,
        4, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 5, 4, 4, 3, 3, 4, 5, 3, 5, 3, 4, 3, 4,
        3, 4, 5, 5, 4, 4, 4, 3, 5, 5, 3, 5, 3, 3, 3, 5, 3, 4, 4, 4, 3, 5, 3, 5,
        5, 3, 3, 4, 5, 5, 3, 4, 3, 3, 3, 4, 3, 3, 3, 5, 4, 5, 3, 3, 5, 3, 5, 5,
        4, 4, 4, 5, 4, 4, 3, 5, 5, 4, 5, 3, 3, 4, 4, 4, 4, 5, 4, 3, 5, 5, 3, 3,
        5, 3, 3, 5, 3, 3, 4, 3, 3, 5, 3, 3, 4, 4, 4, 3, 4, 5, 5, 5, 5, 5, 3, 5,
        3, 5, 3, 5, 3, 5, 4, 3, 4, 4, 4,

In [ ]:
logits.

In [627]:
df = pd.read_pickle('/data/edmitrie/alfa_boosters/df_atr_1_net.pickle')

In [630]:
targ_encoder = pickle.load(open('target_encoder.pickle','rb'))

In [636]:
df['target'] = targ_encoder.inverse_transform(df['target'].astype(int))

In [633]:
df['target'].unique()

array([7, 5, 10, 3, 2, 8, 0, 1, 6, 4, 9], dtype=object)